## THE Peer Finder

### 0 - Import the usual packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

### 1 - Read various SQL tables

In [2]:
import psycopg2

conn_string = open("../PostgresConfig.txt", "r")

con = psycopg2.connect(conn_string.read())

# con = psycopg2.connect(database="wurdata", 
#                        host='wur-data.mgmt.thewurcloud.com', 
#                        user="xxx", 
#                        password="8kbW7U9k", 
#                        port = 5432)

Student data

In [3]:
query = """ 

SELECT A.id,
       the_name,
       foundation_year,
       year,
       subject,
       applicable,
       students_total
       
FROM wur.ds_data_collection_portal as A

LEFT JOIN wur.ref_institution_latest_vw as B

ON A.id = B.id

WHERE (year = 2016 and subject != 'Overall')

"""

In [4]:
df = pd.read_sql_query(query, con)

# .groupby(by=['id', 'foundation_year']).applicable.sum()

WUR score

In [5]:
query = """

SELECT  id, score_number/100 as WUR_score, teaching_score/100 as teaching_score, research_score/100 as research_score, citation_score/100 as citation_score, industry_score/100 as industry_score, international_score/100 as international_score

FROM wur.rnk_wur_2019_latest_vw

WHERE subject = 'Overall'

"""


In [6]:
df1= pd.read_sql_query(query, con).fillna(0)


In [7]:
df1.head()

,id,wur_score,teaching_score,research_score,citation_score,industry_score,international_score
0,i-92611101,0.960,0.918,0.995,0.991,0.670,0.963
1,i-67516113,0.948,0.921,0.988,0.971,0.529,0.943
2,i-35756672,0.947,0.936,0.968,0.999,0.646,0.793
3,i-68334672,0.942,0.919,0.927,0.999,0.876,0.890
4,i-20537945,0.941,0.945,0.972,0.992,0.882,0.623


Reputation score

In [8]:
query = """

SELECT id, score_overall/100 as rep_score

FROM wur.rnk_reputation_2018_latest_vw

"""

In [9]:
df2= pd.read_sql_query(query, con)


Country

In [10]:
query = """

SELECT id, country_code

FROM wur.ref_institution_location_vw

"""

In [11]:
df3= pd.read_sql_query(query, con)


Pivot the table and make all the subjects into separate columns

In [12]:
df_pivoted=df.pivot_table(values='students_total', index=['id','the_name','foundation_year'], columns='subject').reset_index().set_index('id')

In [13]:
df_pivoted = df_pivoted.merge(df1, how='left', on='id').set_index('id')   

df_pivoted = df_pivoted.merge(df2, how='left', on='id')

df_pivoted = df_pivoted.merge(df3, how='left', on='id')

# needed to display ranked-only universities
df_pivoted = df_pivoted[df_pivoted.wur_score>0]
              

In [14]:
df_pivoted.head()

,id,the_name,foundation_year,Arts and Humanities,Business and Economics,Clinical and Health,Computer Science,Education,Engineering,Law,...,Psychology,Social Sciences,wur_score,teaching_score,research_score,citation_score,industry_score,international_score,rep_score,country_code
0,i-00028182,Tongji University,1907.0,5311.0,4984.0,2469.0,2076.0,NaN,16795.0,586.0,...,NaN,1377.0,0.393,0.375,0.417,0.310,0.984,0.505,0.014,CHN
1,i-00136154,University of Zagreb,1669.0,10830.0,8511.0,4308.0,3101.0,3422.0,11150.0,7608.0,...,302.0,5183.0,0.191,0.175,0.127,0.230,0.370,0.296,0.001,HRV
2,i-00185274,Huaqiao University,1960.0,6493.0,6493.0,NaN,1453.0,NaN,10379.0,1343.0,...,NaN,1580.0,0.172,0.128,0.094,0.205,0.381,0.451,0.000,CHN
3,i-00204694,Lund University,1666.0,4577.0,2974.0,3235.0,NaN,442.0,6472.0,2462.0,...,938.0,4937.0,0.625,0.419,0.532,0.874,0.780,0.773,0.028,SWE
4,i-00215262,University of Bari Aldo Moro,1924.0,4826.0,4560.0,5032.0,1495.0,1323.0,NaN,4241.0,...,865.0,2106.0,0.382,0.173,0.170,0.821,0.374,0.320,0.001,ITA


### 3 - Normalize the indicators

In [15]:
# logging works really well to get a decent bell shape out of a distribution with a 
# bell hape at lower values and a high value tail

df_pivoted['norm_teaching_score'] = np.log(df_pivoted['teaching_score'].fillna(0)+1)
df_pivoted['norm_research_score'] = np.log(df_pivoted['research_score'].fillna(0)+1)

# this is pure distribution hacking, as we all know that reputation is deeply flawed
df_pivoted['norm_rep_score'] = (df_pivoted['rep_score'].fillna(0))**0.2

In [16]:
df_pivoted.head()

,id,the_name,foundation_year,Arts and Humanities,Business and Economics,Clinical and Health,Computer Science,Education,Engineering,Law,...,teaching_score,research_score,citation_score,industry_score,international_score,rep_score,country_code,norm_teaching_score,norm_research_score,norm_rep_score
0,i-00028182,Tongji University,1907.0,5311.0,4984.0,2469.0,2076.0,NaN,16795.0,586.0,...,0.375,0.417,0.310,0.984,0.505,0.014,CHN,0.318454,0.348542,0.425820
1,i-00136154,University of Zagreb,1669.0,10830.0,8511.0,4308.0,3101.0,3422.0,11150.0,7608.0,...,0.175,0.127,0.230,0.370,0.296,0.001,HRV,0.161268,0.119559,0.251189
2,i-00185274,Huaqiao University,1960.0,6493.0,6493.0,NaN,1453.0,NaN,10379.0,1343.0,...,0.128,0.094,0.205,0.381,0.451,0.000,CHN,0.120446,0.089841,0.000000
3,i-00204694,Lund University,1666.0,4577.0,2974.0,3235.0,NaN,442.0,6472.0,2462.0,...,0.419,0.532,0.874,0.780,0.773,0.028,SWE,0.349952,0.426574,0.489138
4,i-00215262,University of Bari Aldo Moro,1924.0,4826.0,4560.0,5032.0,1495.0,1323.0,NaN,4241.0,...,0.173,0.170,0.821,0.374,0.320,0.001,ITA,0.159565,0.157004,0.251189


In [17]:
df_pivoted.set_index('id', inplace=True)

### 4 - Create two new indicators, "concentration" and "STEMness"

In [18]:
subjects = list(df_pivoted.columns)[2:13]

In [19]:
STEM = [
        'Computer Science',
       'Engineering',
       'Life Sciences',
       'Physical Sciences',
        'Psychology'
       ]

df_pivoted['STEMness'] = df_pivoted[STEM].sum(axis=1) / df_pivoted[subjects].sum(axis=1)
    
df_pivoted['concentration'] = 0

for subj in subjects:
    df_pivoted['concentration'] = df_pivoted['concentration'] + df_pivoted[subj].fillna(0)**2
    
df_pivoted['concentration'] = np.sqrt(df_pivoted['concentration'] / df_pivoted[subjects].sum(axis=1)**2)

# df_pivoted['concentration_norm']=((df_pivoted['concentration']-df_pivoted['concentration'].min())*df_pivoted['concentration'].max()/ (df_pivoted['concentration']-df_pivoted['concentration'].min()).max())

# df_pivoted = df_pivoted[df_pivoted.score_number>0]

In [20]:
features = ['STEMness', 'concentration_norm', 'norm_teaching_score', 'norm_research_score', 'citation_score', 'industry_score', 'international_score', 'norm_rep_score']

In [21]:
def find_most_similar_to(ix, df):
    
# pandas method that subtract the row characterised by the id from all rows in the tables
        
    dist = df.sub(df.loc[ix].values, axis='columns')
            
#     dist['subj'] = (1-subjects_match(df_in, ix).new)/10

# calcultates the sum of the differences squared
            
    dist['dist'] = np.sqrt((dist**2).sum(axis=1))
    
# produces a sorted list id, distance
    
    dist=dist.sort_values('dist').iloc[1:]
    
#     print(dist.iloc[:5])
    
    return dist.dist

In [22]:
import ipywidgets as widgets
from IPython.display import display

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
def find_5_peers(country_code, name, features):
    
    df_in = df_pivoted[df_pivoted.country_code == country_code]
    
    index = df_in[df_in.the_name==name].index[0]
    
    df_in =df_in.loc[:,features].fillna(0)
    
    scaled_df = pd.DataFrame(StandardScaler().fit_transform(df_in), index=df_in.index)
    
    print(find_most_similar_to(index, scaled_df).to_frame().join(df_pivoted['the_name']).head(5))

In [25]:
country_widget = widgets.Dropdown(options=sorted(list(df_pivoted.country_code.unique())))
name_widget =  widgets.Dropdown(options=sorted(list(df_pivoted[df_pivoted.country_code==country_widget.value].the_name)))
features_widget = widgets.SelectMultiple(options=features, value=['norm_rep_score'])

def update_name_widget(*args):
    name_widget.options = sorted(list(df_pivoted[df_pivoted.country_code==country_widget.value].the_name))
    
country_widget.observe(update_name_widget, 'value')

display(widgets.interactive(find_5_peers, 
                            country_code = country_widget, 
                            name = name_widget, 
                            features = features_widget))


interactive(children=(Dropdown(description='country_code', options=('ARE', 'ARG', 'AUS', 'AUT', 'BEL', 'BGR', …